In [1]:
import os
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
import re
from bs4 import BeautifulSoup
import requests

In [2]:
round_ = '17'
url = f'https://www.sportsbet.com.au/betting/australian-rules/afl/round-{round_}'
url

'https://www.sportsbet.com.au/betting/australian-rules/afl/round-17'

In [3]:
soup = BeautifulSoup(requests.get(url).text, 'html.parser')

In [4]:
tmp = soup.find_all('script')

In [5]:
game_urls = []
for i in tmp[:9]:
    try:
        url_find = re.compile(r'https://www.sportsbet.com.au/betting/australian-rules/afl/(.*?)\-\d{7}')
        game_url = url_find.search(i.text)
        game_urls.append(game_url.group())
    except:
        continue
game_urls

['https://www.sportsbet.com.au/betting/australian-rules/afl/brisbane-v-west-coast-7454971',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/greater-western-sydney-v-hawthorn-7454980',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/st-kilda-v-melbourne-7454981',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/port-adelaide-v-gold-coast-7455001',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/geelong-v-north-melbourne-7455010',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/essendon-v-adelaide-7455011',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/fremantle-v-carlton-7455012']

In [6]:
r = re.compile(r'\/(\w*\-?\w*\-?\w*)\-v\-(\w*\-?\w*\-?\w*)\-')
game_list = []
for game in game_urls:
    game_list.append(r.findall(game))

In [7]:
game_list

[[('brisbane', 'west-coast')],
 [('greater-western-sydney', 'hawthorn')],
 [('st-kilda', 'melbourne')],
 [('port-adelaide', 'gold-coast')],
 [('geelong', 'north-melbourne')],
 [('essendon', 'adelaide')],
 [('fremantle', 'carlton')]]

In [8]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.set_window_size(700,800)

[WDM] - Downloading: 100%|█████████████████| 8.29M/8.29M [00:00<00:00, 12.2MB/s]


In [9]:
disposals = ['To Get 15 or More Disposals',
            'To Get 20 or More Disposals',
            'To Get 25 or More Disposals',
            'To Get 30 or More Disposals',]
output = pd.DataFrame({},columns = {'teams','round','Player', 'disposals', 'odds'})

In [10]:
for game, game_url in zip(game_list, game_urls):
    driver.get(game_url)
    try:
        first_click = driver.find_element_by_xpath('(//*[@class="size14_f7opyze medium_f1wf24vo"])')
        first_click.click()
        
        for i in range(1,7):
            if driver.find_element_by_xpath(f'(//*[@class="size14_f7opyze medium_f1wf24vo"])[{i}]').text == 'Disposal Markets':
                disposal_market = driver.find_element_by_xpath(f'(//*[@class="size14_f7opyze medium_f1wf24vo"])[{i}]')
        
        
        disposal_market.click()
        time.sleep(3)
        
        for i in range(5,50):
            try:
                tmp = driver.find_element_by_xpath(f'(//*[@class="size14_f7opyze bold_f1au7gae"])[{i}]').text
            except:
                continue
            if tmp in disposals:

                dis = driver.find_element_by_xpath(f'(//*[@class="size14_f7opyze bold_f1au7gae"])[{i}]')

                dis.click()

                player = driver.find_element_by_xpath(f'(//*[@class="outcomeContainer_f1wc7xgg outcomeDetailsFirst_f162fqwy"])').text.split('\n')
                print(player)
                new_row = {'Player': player[0].lower(), 'teams': f'{game[0][0]} v {game[0][1]}', 'round':f'R{round_}', 'disposals':tmp,'odds':player[-1]}
                output.loc[len(output)] = new_row

                for j in range(1,50):
                    try:

                        tmp1 = driver.find_element_by_xpath(f'(//*[@class="outcomeContainer_f1wc7xgg"])[{j}]').text.split('\n')
                        print(tmp1)
                        new_row = {'Player': tmp1[0].lower(), 'teams': f'{game[0][0]} v {game[0][1]}', 'round':f'R{round_}', 'disposals':tmp,'odds':tmp1[-1]}
                        output.loc[len(output)] = new_row
                    except:
                        continue
                
                dis.click()
                time.sleep(2)
            
    except:
        continue

['Jarryd Lyons', 'SUS']
['Dayne Zorko', '1.13']
['Zac Bailey', '1.20']
['Keidean Coleman', '1.25']
['Conor McKenna', '1.26']
['Joe Daniher', '1.34']
['Deven Robertson', '1.36']
['Darcy Wilmot', '1.43']
['Jaspa Fletcher', '1.50']
['Oscar McInerney', '1.67']
['Cam Rayner', '1.70']
['Harris Andrews', '1.85']
['Ryan Lester', '1.96']
['Brandon Starcevich', '2.25']
['Jack Gunston', '3.80']
['Eric Hipwood', '4.00']
['Alex Witherden', '1.09']
['Liam Duggan', '1.11']
['Dom Sheed', '1.13']
['Jack Petruccelle', '1.36']
['Jayden Hunt', '1.43']
['Reuben Ginbey', '1.50']
['Andrew Gaff', '1.61']
['Elijah Hewett', '1.65']
['Brady Hough', '1.95']
["Xavier O'Neill", '1.95']
['Bailey J. Williams (WCE)', '1.96']
['Tom Barrass', '1.96']
['Noah Long', '2.40']
['Jamie Cripps', '2.90']
['Hugh McCluggage', '1.14']
['Jarryd Lyons', '1.20']
['Will Ashcroft', '1.20']
['Dayne Zorko', '1.73']
['Zac Bailey', '2.15']
['Conor McKenna', '2.25']
['Keidean Coleman', '2.25']
['Darcy Wilmot', '2.60']
['Joe Daniher', '2.65'

['Callum Wilkie', 'Last 5: 16-15-17-23-21', '1.41']
['Marcus Windhager', 'Last 5: 20-12-20-15-4', '1.44']
['Jade Gresham', 'Last 5: 21-11-16-18-15', '1.47']
['Liam Stocker', 'Last 5: 9-18-13-13-23', '1.49']
['Jimmy Webster', 'Last 5: 15-15-13-19-16', '1.55']
['Dougal Howard', 'Last 5: 18-12-14-13-16', '1.93']
['Mitchito Owens', 'Last 5: N/A', '1.98']
['Mattaes Phillipou', 'Last 5: N/A', '2.35']
['Dan Butler', 'Last 5: 10-10-13-10-14', '3.75']
['Jack Viney', 'Last 5: 41-25-32-20-16', 'SUS']
['Ed Langdon', 'Last 5: 24-21-14-25-19', '1.10']
['James Jordon', 'Last 5: N/A', '1.10']
['Steven May', 'Last 5: 22-16-18-18-8', '1.13']
['Trent Rivers', 'Last 5: 17-25-23-24-13', '1.13']
['Tom Sparrow', 'Last 5: 26-19-19-15-18', '1.18']
['Jake Bowey', 'Last 5: 19-17-18-10-14', '1.22']
['Taj Woewodin', 'Last 5: N/A', '1.35']
['Max Gawn', 'Last 5: 20-15-15-16-15', '1.44']
['Jake Lever', 'Last 5: 19-15-10-17-7', '1.54']
['Alex Neal-Bullen', 'Last 5: 19-16-14-11-18', '1.66']
['Brodie Grundy', 'Last 5: 1

['Wil Powell', 'Last 5: 27-19-18-17-16', '2.25']
['Brandon Ellis', 'Last 5: 18-16-14-12-11', '2.55']
['David Swallow', 'Last 5: 15-10-18-16-19', '2.65']
['Ben Ainsworth', 'Last 5: 8-27-22-14-14', '3.35']
['Jarrod Witts', 'Last 5: 11-14-10-6-19', '3.40']
['Bailey Humphrey', 'Last 5: N/A', '3.50']
['Charlie Ballard', 'Last 5: 18-12-17-9-14', '4.90']
['Jack Lukosius', 'Last 5: 14-11-18-16-12', '5.90']
['Zak Butters', 'Last 5: 25-23-31-26-32', '1.42']
['Connor Rozee', 'Last 5: 23-25-23-29-27', '1.64']
['Dan Houston', 'Last 5: 32-31-17-20-25', '2.25']
['Ollie Wines', 'Last 5: 13-28-18-24-21', '2.55']
['Kane Farrell', 'Last 5: 25-15-17-14-23', '6.25']
['Jason Horne-Francis', 'Last 5: 10-13-13-18-23', '6.50']
['Dylan Williams', 'Last 5: 22-19-16-19-13', '7.00']
['Travis Boak', 'Last 5: 8-13-26-12-12', '7.50']
['Willem Drew', 'Last 5: 18-14-14-17-10', '12.50']
['Jeremy Finlayson', 'Last 5: 14-10-8-17-17', '14.00']
['Ryan Burton', 'Last 5: 16-13-13-7-10', '15.00']
['Darcy Byrne-Jones', 'Last 5:

['Jake Stringer', 'Last 5: 13-16-18-17-18', '1.55']
['Sam Durham', 'Last 5: 14-15-17-15-10', '1.66']
['Archie Perkins', 'Last 5: 12-16-18-18-15', '1.69']
['Jayden Laverde', 'Last 5: 15-12-12-11-15', '1.93']
['Kyle Langford', 'Last 5: 16-7-11-14-15', '1.96']
['Jake Kelly', 'Last 5: 13-12-14-11-11', '2.25']
['Will Snelling', 'Last 5: 14-13-14-12-14', '2.55']
['Brandon Zerk-Thatcher', 'Last 5: 9-8-10-12-12', '3.60']
['Andrew Phillips', 'Last 5: 12-11-7-8-1', '3.90']
['Matt Guelfi', 'Last 5: 8-12-14-14-12', '3.90']
['Mitch Hinge', 'Last 5: 26-23-18-16-1', '1.13']
['Brodie Smith', 'Last 5: 24-19-18-26-14', '1.19']
['Rory Sloane', 'Last 5: 30-19-19-17-14', '1.22']
['Wayne Milera', 'Last 5: 18-18-21-20-22', '1.22']
['Chayce Jones', 'Last 5: 16-23-21-17-17', '1.32']
['Josh Rachele', 'Last 5: 23-11-15-12-15', '1.42']
['Lachlan Sholl', 'Last 5: 21-11-17-15-20', '1.43']
['Izak Rankine', 'Last 5: 17-21-20-13-23', '1.44']
['Jake Soligo', 'Last 5: 16-11-15-9-18', '1.53']
['Lachlan Murphy', 'Last 5: 

['Charlie Curnow', 'Last 5: 13-15-17-11-18', '4.60']
['Alex Cincotta', 'Last 5: N/A', '5.60']
['Harry McKay', 'Last 5: 19-10-14-12-8', '8.00']
['Caleb Serong', 'Last 5: 38-32-32-31-26', '1.17']
['Andrew Brayshaw', 'Last 5: 29-29-29-29-29', '1.22']
['Liam Henry', 'Last 5: 33-32-17-10-13', '1.96']
['Luke Ryan', 'Last 5: 24-31-22-16-22', '2.18']
['Hayden Young', 'Last 5: 21-17-20-21-27', '2.95']
['James Aish', 'Last 5: 17-26-24-22-12', '3.70']
["Jaeger O'Meara", 'Last 5: 21-21-15-24-26', '3.80']
['Matt Johnson', 'Last 5: N/A', '3.90']
['Brennan Cox', 'Last 5: 31-13-19-13-20', '5.10']
['Luke Jackson', 'Last 5: 8-17-17-23-19', '9.75']
['Sean Darcy', 'Last 5: 18-14-6-8-18', '12.50']
['Lachie Schultz', 'Last 5: 18-10-17-17-10', '16.00']
['Neil Erasmus', 'Last 5: 2-12-10-15-16', '17.00']
['Sam Walsh', 'Last 5: 25-29-35-22-31', '1.25']
['Adam Cerra', 'Last 5: 27-27-31-30-24', '1.50']
['Sam Docherty', 'Last 5: 26-32-27-26-25', '1.56']
['Patrick Cripps', 'Last 5: 28-27-19-23-19', '1.64']
['Blake 

In [52]:
output

,disposals,teams,odds,Player,round
0,To Get 15 or More Disposals,western-bulldogs v collingwood,SUS,caleb daniel,R17
1,To Get 15 or More Disposals,western-bulldogs v collingwood,1.15,ed richards,R17
2,To Get 15 or More Disposals,western-bulldogs v collingwood,1.15,tim english,R17
3,To Get 15 or More Disposals,western-bulldogs v collingwood,1.44,bailey williams (wb),R17
4,To Get 15 or More Disposals,western-bulldogs v collingwood,1.95,caleb poulter,R17
...,...,...,...,...,...
838,To Get 30 or More Disposals,fremantle v carlton,6.00,blake acres,R17
839,To Get 30 or More Disposals,fremantle v carlton,9.25,matthew kennedy,R17
840,To Get 30 or More Disposals,fremantle v carlton,10.00,adam saad,R17
841,To Get 30 or More Disposals,fremantle v carlton,11.00,nic newman,R17


In [53]:
output['home'] = output['teams'].apply(lambda x: x.split(' v ')[0])
output['away'] = output['teams'].apply(lambda x: x.split(' v ')[1])

In [54]:
team_mapping = {'richmond':'RCH','geelong':'GEE',
                'north-melbourne':'NTH','port-adelaide':'POR',
                'gold-coast':'GCS','melbourne':'MEL',
                'greater-western-sydney':'GWS','hawthorn':'HAW',
                'carlton':'CAR','brisbane':'BRL',
                'collingwood':'COL','adelaide':'ADE',
                'fremantle':'FRE','western-bulldogs':'WBD',
                'sydney':'SYD','west-coast':'WCE',
                'st-kilda':'STK','essendon':'ESS'}

In [55]:
for key in team_mapping.keys():
     output.loc[output['home']==key, 'home'] = team_mapping[key]
for key in team_mapping.keys():
    output.loc[output['away']==key, 'away'] = team_mapping[key]     

In [56]:
output

,disposals,teams,odds,Player,round,home,away
0,To Get 15 or More Disposals,western-bulldogs v collingwood,SUS,caleb daniel,R17,WBD,COL
1,To Get 15 or More Disposals,western-bulldogs v collingwood,1.15,ed richards,R17,WBD,COL
2,To Get 15 or More Disposals,western-bulldogs v collingwood,1.15,tim english,R17,WBD,COL
3,To Get 15 or More Disposals,western-bulldogs v collingwood,1.44,bailey williams (wb),R17,WBD,COL
4,To Get 15 or More Disposals,western-bulldogs v collingwood,1.95,caleb poulter,R17,WBD,COL
...,...,...,...,...,...,...,...
838,To Get 30 or More Disposals,fremantle v carlton,6.00,blake acres,R17,FRE,CAR
839,To Get 30 or More Disposals,fremantle v carlton,9.25,matthew kennedy,R17,FRE,CAR
840,To Get 30 or More Disposals,fremantle v carlton,10.00,adam saad,R17,FRE,CAR
841,To Get 30 or More Disposals,fremantle v carlton,11.00,nic newman,R17,FRE,CAR


In [57]:
output.to_csv('/Users/caseyh/Desktop/footyfootyfooty/sportsbet_odds/test_disposal_markets_r17_sat.csv', index = False)

In [13]:
from datetime import datetime

In [17]:
datetime.now()

datetime.datetime(2023, 7, 8, 13, 29, 21, 552976)